In [ ]:
import os
import requests
import pandas as pd
from datetime import datetime, timedelta

In [ ]:
# Data Collection From the JGSEE Best Station Database
# Maximum query one year at a time
start_date = '2021-01-01'
end_date = '2021-12-31'

response = requests.get(f'http://localhost:3000/api/floor7?sort=timestamp&keys=date>={start_date},date<={end_date}')
df_database = pd.DataFrame(response.json())
df_database['timestamp'] = df_database['timestamp'].apply(lambda x: (datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ') + timedelta(hours = 7)).strftime("%Y-%m-%dT%H:%M:%S+07:00"))
df_database['date'] = df_database['date'].apply(lambda x: x.split('T')[0])
df_database.rename(columns={'temperature': 'Temperature'}, inplace=True)
for date in df_database['date'].unique():
    df_database.query(f'date == "{date}"').to_csv(f'rawdata/{date}.csv', header=True,index=False)

In [ ]:
# Data Collection From CSV File
header = ["timestamp", "date", "time", "Blank", "Eeg", "Eed", "Eeb", "EeN", "EeE", "EeS", "EeW", 
            "Evg", "Evd", "Evb", "EvN", "EvE", "EvS", "EvW", "lz", "Thermopile", "Thermistor",
            "Windspeed", "Winddirection", "RH", "Temperature"]

files = [f for f in os.listdir('stationdata/') if not f.startswith('.')]
files = sorted(files)

for file in files:
    file_name = file.split('.')[0]
    file_name = file_name.split('-')
    date = datetime(2000+int(file_name[2]), int(file_name[1]), int(file_name[0]))
    date_name = date.strftime('%Y-%m-%d')
    df_station = pd.read_csv(f'stationdata/{file}',names=header)
    df_station['time'] = df_station['time'].apply(lambda x: f'0{x}' if len(x.split(':')[0])==1 else x)
    df_station['date'] = date_name
    df_station['timestamp'] = df_station['date']+'T'+df_station['time']+'+07:00'
    df_station.to_csv(f'rawdata/{date_name}.csv', header=True,index=False)
    print(date_name)